In [1]:
# -*- coding:UTF-8 -*-

    
from corpus import *
from vsm import *
from vsm.spatial import KL_div


k = topic_range[0]
v = lda_v[k]
c.words

# train the model and create a TfViewer object
tf = TF(c, context_type)
tf.train()
tf_v = TfViewer(c, tf)

# print the most frequent terms in the document
# remember that IPython automatically prints the last cell of a document
tf_v.coll_freqs()




#v.topics()

#v.doc_topics(v.labels[:3])


#v.dist(v.labels[0], v.labels[1])
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.random import randn
import numpy as np
import matplotlib.font_manager as fm
myfont = fm.FontProperties(fname='/home/hongliang/Downloads/msyh.ttf/msyh.ttf')
import sys
reload(sys)
kongzi = []
mengzi = []
xunzi =[]

count = 0
for label in v.labels[:]:
    if '孟子' in label:
        mengzi.append(count)
    elif '论语' in label:
        kongzi.append(count)
    elif '荀子' in label:
        xunzi.append(count)
    count += 1



Running from notebook, using serial load function.
[20, 40, 60, 80, 100]
/home/hongliang/inpho/kmx/models/kmx-freq5-freq5-N2523342-LDA-K{0}-document-1000.npz
Loading LDA data from /home/hongliang/inpho/kmx/models/kmx-freq5-freq5-N2523342-LDA-K20-document-1000.npz


In [2]:
def similarity(c1,c2):
    A = np.zeros((len(c1),len(c2)))
    for m in range(len(c1)):
        for n in range(len(c2)):
            A[m][n]= v.dist(v.labels[c1[m]],v.labels[c2[n]]) 
    print "while k quals",k,',','JSD goes',np.mean(A)
    return np.mean(A)

def kl_similarity(c1,c2):
    A = np.zeros((len(c1),len(c2)))
    for m in range(len(c1)):
        for n in range(len(c2)):
            A[m][n]= v.dist(v.labels[c1[m]],v.labels[c2[n]], dist_fn=KL_div) 
    print "while k quals",k,',','first to second KLD goes',np.mean(A)
    
    B = np.zeros((len(c1),len(c2)))
    for m in range(len(c1)):
        for n in range(len(c2)):
            B[m][n]= v.dist(v.labels[c2[n]],v.labels[c1[m]], dist_fn=KL_div) 
    print "while k quals",k,',','second to first KLD goes',np.mean(B)
        
    
    
    return np.mean(A),np.mean(B)


In [3]:
print ""

In [4]:
for i in range(5):
    k = topic_range[i]
    v = lda_v[k]
    print 'kongzi mengzi JSD'
    similarity(kongzi,mengzi)
    print 'kongzi xunzi JSD'
    similarity(kongzi,xunzi)
    print 'xunzi mengzi JSD'
    similarity(xunzi,mengzi)
    print 'kongzi mengzi KLD'
    kl_similarity(kongzi,mengzi)
    print 'kongzi xunzi KLD'
    kl_similarity(kongzi,xunzi)
    print 'xunzi mengzi KLD'
    kl_similarity(xunzi,mengzi)
    
for i in range(5):
    k = topic_range[i]
    v = lda_v[k]    
    perplexity_num = []
    documents = v.corpus.view_contexts('document')
    for doc_id,corpus in zip(all_ids,documents):
        theta = v.doc_topic_matrix(doc_id)
        perplexity_num.append(perplexity(theta,corpus,v))
    perplexity_num = np.array(perplexity_num)
    print k
    print np.mean(perplexity_num)

NameError: name 'perplexity' is not defined

### original with bug 
TypeError: invalid type promotion

In [5]:
from corpus import *
import numpy as np

def perplexity(theta, corpus, v):
    a = (v.phi[corpus] * theta).sum(axis=1)
    a = np.log2(a).sum()
    return a / len(corpus)

v = lda_v[80]

# v.aggregate_doc_topics returns an ndarray instead of just values.
theta = v.aggregate_doc_topics(all_ids)
# theta = theta[np.argsort(theta['i'])]
# theta = np.array(theta['value'])
print theta
corpus = v.corpus.corpus
P = perplexity(theta, corpus, v)
print P

TypeError: invalid type promotion

## my method

In [12]:
from corpus import *
import numpy as np

def perplexity(theta, corpus, v):
   a = (v.phi[corpus] * theta).sum(axis=1)
   a = np.log2(a).sum()
   return a / len(corpus)

v = lda_v[80]
theta = v.aggregate_doc_topics(all_ids)
TMP = []
for a in theta:
   TMP.append(a[1]) 
theta = np.array(TMP)
print theta
corpus = v.corpus.corpus
P = perplexity(theta, corpus, v)
print P

## each to whole

In [17]:
def perplexity(theta, corpus, v):
    a = (v.phi[corpus] * theta).sum(axis=1)
    a = np.log2(a).sum()
    return a / len(corpus)

v = lda_v[80]    
perplexity_num = []
theta_all = []
documents = v.corpus.view_contexts('document')
for doc_id,corpus in zip(all_ids,documents):
    theta = v.doc_topic_matrix(doc_id)
    print theta
    theta_all.append(theta)
    perplexity_num.append(perplexity(theta,corpus,v))
perplexity_num = np.array(perplexity_num)
theta_all = np.array(theta_all)
print 'each document way',np.mean(perplexity_num)


### Jaimie's Final method

In [13]:
from corpus import *
import numpy as np

def perplexity(theta, corpus, v):
    a = (v.phi[corpus] * theta).sum(axis=1)
    a = np.log2(a).sum()
    return a / len(corpus)

v = lda_v[80]
corpus = v.corpus.corpus

# v.aggregate_doc_topics returns an ndarray instead of just values.
theta = v.aggregate_doc_topics(all_ids)
print 'theta 1 step',theta
theta = theta[np.argsort(theta['i'])]
print 'theat2 step',theta
theta = np.array(theta['value'])
print 'theta3 step',theta

P = perplexity(theta, corpus, v)
print 'whole corpus',P
